***GENERATED CODE FOR prep PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def replaceValues(df, functionsData, applyOn):
    for columnData in applyOn:
        for functionData in functionsData:
            if functionData['replaceType'] == 'value':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, functionData['ReplaceWith'])
            elif functionData['replaceType'] == 'none':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", minValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", maxValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        "", functionData['ReplaceWith'])
                    df[columnData['columnName']] = df[columnData['columnName']].fillna(
                        functionData['ReplaceWith'])
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'job', 'transformation_label': 'String Indexer'}], 'feature': 'job', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'admin.', 'max': 'unknown', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'job'}, {'feature_label': 'job', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('job')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'marital', 'transformation_label': 'String Indexer'}], 'feature': 'marital', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'divorced', 'max': 'single', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'marital'}, {'feature_label': 'marital', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('marital')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'education', 'transformation_label': 'String Indexer'}], 'feature': 'education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'primary', 'max': 'unknown', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'education'}, {'feature_label': 'education', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('education')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'default', 'transformation_label': 'String Indexer'}], 'feature': 'default', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'default'}, {'feature_label': 'default', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('default')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'housing', 'transformation_label': 'String Indexer'}], 'feature': 'housing', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'housing'}, {'feature_label': 'housing', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('housing')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'loan', 'transformation_label': 'String Indexer'}], 'feature': 'loan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'loan'}, {'feature_label': 'loan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('loan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'contact', 'transformation_label': 'String Indexer'}], 'feature': 'contact', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'cellular', 'max': 'unknown', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'contact'}, {'feature_label': 'contact', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('contact')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'month', 'transformation_label': 'String Indexer'}], 'feature': 'month', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'apr', 'max': 'sep', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'month'}, {'feature_label': 'month', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('month')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'poutcome', 'transformation_label': 'String Indexer'}], 'feature': 'poutcome', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'failure', 'max': 'unknown', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'poutcome'}, {'feature_label': 'poutcome', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('poutcome')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'deposit', 'transformation_label': 'String Indexer'}], 'feature': 'deposit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'deposit'}, {'feature_label': 'deposit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('deposit')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run prepHooks.ipynb
try:
	#sourcePreExecutionHook()

	bankdepositanalysistrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/TrainData/BankDepositAnalysisTrain.csv', 'filename': 'BankDepositAnalysisTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(bankdepositanalysistrain)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run prepHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(bankdepositanalysistrain,spark,json.dumps( {"url": "/FileStore/platform/TrainData/BankDepositAnalysisTrain.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/TrainData/BankDepositAnalysisTrain.csv", "data_source": "hdfs", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/61024a89beb57c3b89688d731627549919/0part.csv", "requestRatio": 22.0, "totalRows": 11138, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 17, "numberOfRows": 11138, "duplicateRowCount": 0, "stats": [{"column": "age", "alias": "age", "generated": 0, "type": "real", "max": 95.0, "min": 18.0, "mean": 41.22876638534746, "missing": 0.0, "stddev": 11.91}, {"column": "job", "alias": "job", "generated": 0, "type": "String", "max": "unknown", "min": "admin.", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "marital", "alias": "marital", "generated": 0, "type": "String", "max": "single", "min": "divorced", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "education", "alias": "education", "generated": 0, "type": "String", "max": "unknown", "min": "primary", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "default", "alias": "default", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "balance", "alias": "balance", "generated": 0, "type": "real", "max": 81204.0, "min": -6847.0, "mean": 1528.8028371341354, "missing": 0.0, "stddev": 3227.29}, {"column": "housing", "alias": "housing", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "loan", "alias": "loan", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "contact", "alias": "contact", "generated": 0, "type": "String", "max": "unknown", "min": "cellular", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "day", "alias": "day", "generated": 0, "type": "real", "max": 31.0, "min": 1.0, "mean": 15.653438678398276, "missing": 0.0, "stddev": 8.42}, {"column": "month", "alias": "month", "generated": 0, "type": "String", "max": "sep", "min": "apr", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "duration", "alias": "duration", "generated": 0, "type": "real", "max": 3881.0, "min": 2.0, "mean": 372.29412820973243, "missing": 0.0, "stddev": 347.4}, {"column": "campaign", "alias": "campaign", "generated": 0, "type": "real", "max": 63.0, "min": 1.0, "mean": 2.510235230741605, "missing": 0.0, "stddev": 2.72}, {"column": "pdays", "alias": "pdays", "generated": 0, "type": "real", "max": 854.0, "min": -1.0, "mean": 51.352846112407974, "missing": 0.0, "stddev": 108.8}, {"column": "previous", "alias": "previous", "generated": 0, "type": "real", "max": 58.0, "min": 0.0, "mean": 0.8326449991021727, "missing": 0.0, "stddev": 2.29}, {"column": "poutcome", "alias": "poutcome", "generated": 0, "type": "String", "max": "unknown", "min": "failure", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "deposit", "alias": "deposit", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": ""}]}, "HasBasicStats": 1, "functionsApplied": [{"functionName": "replaceBy", "applyOn": [{"columnName": "default", "type": "String", "min": "no", "max": "yes", "mean": "-"}], "functionsData": [{"replaceType": "none", "toReplace": "", "asNewColumn": 0, "newColumnName": "", "replaceValueType": "min", "ReplaceWith": ""}]}], "functionChanges": [{"columnName": "default", "functionName": "Replace Outliers", "Type": "String", "Parameters": [{"replaceType": "none", "toReplace": "", "asNewColumn": 0, "newColumnName": "", "replaceValueType": "min", "ReplaceWith": ""}]}], "fileheader": [{"field": "age", "alias": "age", "generated": 0, "position": 1, "type": "real"}, {"field": "job", "alias": "job", "generated": 0, "position": 2, "type": "String"}, {"field": "marital", "alias": "marital", "generated": 0, "position": 3, "type": "String"}, {"field": "education", "alias": "education", "generated": 0, "position": 4, "type": "String"}, {"field": "default", "alias": "default", "generated": 0, "position": 5, "type": "String"}, {"field": "balance", "alias": "balance", "generated": 0, "position": 6, "type": "real"}, {"field": "housing", "alias": "housing", "generated": 0, "position": 7, "type": "String"}, {"field": "loan", "alias": "loan", "generated": 0, "position": 8, "type": "String"}, {"field": "contact", "alias": "contact", "generated": 0, "position": 9, "type": "String"}, {"field": "day", "alias": "day", "generated": 0, "position": 10, "type": "real"}, {"field": "month", "alias": "month", "generated": 0, "position": 11, "type": "String"}, {"field": "duration", "alias": "duration", "generated": 0, "position": 12, "type": "real"}, {"field": "campaign", "alias": "campaign", "generated": 0, "position": 13, "type": "real"}, {"field": "pdays", "alias": "pdays", "generated": 0, "position": 14, "type": "real"}, {"field": "previous", "alias": "previous", "generated": 0, "position": 15, "type": "real"}, {"field": "poutcome", "alias": "poutcome", "generated": 0, "position": 16, "type": "String"}, {"field": "deposit", "alias": "deposit", "generated": 0, "position": 17, "type": "String"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run prepHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "41.16", "stddev": "11.73", "min": "20.0", "max": "93.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "age"}, {"transformationsData": [{"feature_label": "job", "transformation_label": "String Indexer"}], "feature": "job", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "admin.", "max": "unknown", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "job"}, {"transformationsData": [{"feature_label": "marital", "transformation_label": "String Indexer"}], "feature": "marital", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "divorced", "max": "single", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "marital"}, {"transformationsData": [{"feature_label": "education", "transformation_label": "String Indexer"}], "feature": "education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "primary", "max": "unknown", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "education"}, {"transformationsData": [{"feature_label": "default", "transformation_label": "String Indexer"}], "feature": "default", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "default"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "balance", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1538.77", "stddev": "2458.81", "min": "-637.0", "max": "17964.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "balance"}, {"transformationsData": [{"feature_label": "housing", "transformation_label": "String Indexer"}], "feature": "housing", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "housing"}, {"transformationsData": [{"feature_label": "loan", "transformation_label": "String Indexer"}], "feature": "loan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "loan"}, {"transformationsData": [{"feature_label": "contact", "transformation_label": "String Indexer"}], "feature": "contact", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "cellular", "max": "unknown", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "contact"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "day", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "15.98", "stddev": "8.46", "min": "1.0", "max": "31.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "day"}, {"transformationsData": [{"feature_label": "month", "transformation_label": "String Indexer"}], "feature": "month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "apr", "max": "sep", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "duration", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "392.49", "stddev": "347.99", "min": "9.0", "max": "2016.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "duration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.71", "stddev": "3.87", "min": "1.0", "max": "63.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "campaign"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pdays", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "55.3", "stddev": "112.54", "min": "-1.0", "max": "774.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "pdays"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.74", "stddev": "1.72", "min": "0.0", "max": "11.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "previous"}, {"transformationsData": [{"feature_label": "poutcome", "transformation_label": "String Indexer"}], "feature": "poutcome", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "failure", "max": "unknown", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "poutcome"}, {"transformationsData": [{"feature_label": "deposit", "transformation_label": "String Indexer"}], "feature": "deposit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "deposit"}, {"feature": "job_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "2.65", "stddev": "2.59", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "job_stringindexer_transfo..."}, {"feature": "marital_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.6", "stddev": "0.72", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "marital_stringindexer_tra..."}, {"feature": "education_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.77", "stddev": "0.87", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "education_stringindexer_t..."}, {"feature": "default_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "default_stringindexer_tra..."}, {"feature": "housing_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.49", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "housing_stringindexer_tra..."}, {"feature": "loan_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.11", "stddev": "0.32", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "loan_stringindexer_transf..."}, {"feature": "contact_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.31", "stddev": "0.58", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "contact_stringindexer_tra..."}, {"feature": "month_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "3.22", "stddev": "3.06", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "month_stringindexer_trans..."}, {"feature": "poutcome_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.47", "stddev": "0.86", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "poutcome_stringindexer_tr..."}, {"feature": "deposit_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.46", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "deposit_stringindexer_tra..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run prepHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=XG-Boost-classifier(autofe, ["age", "job_stringindexer", "marital_stringindexer", "education_stringindexer", "default_stringindexer", "balance", "housing_stringindexer", "loan_stringindexer", "contact_stringindexer", "day", "month_stringindexer", "duration", "campaign", "pdays", "previous", "poutcome_stringindexer"], "deposit_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
